In [10]:
from TPNN.tools.perceptron import*
import numpy as np
import random as rd
import numpy.random as rand

### Test net

In [2]:
net = Net()
net.insert_layer(0, 2, ident)
net.insert_layer(1, 5, sigmoid)
net.insert_layer(2, 1, ident)
print(net.layers_count)
net.init_weights(1, np.array([[1,1,1,1,1], [1,1,1,1,1]]))
net.init_weights(2, np.array([[1,1,1,1,1]]).transpose())
net.init_biases(2, np.array([[0]]))
net.init_biases(1, np.array([[1,1,1,1,1]]))

net.print_net_config()

3
layers=3
[0]
 size=2
 act_function=<function ident at 0x068C1220>
 activations=[[0. 0.]]
 biases=[[0. 0.]]
 weights:
     empty
--------------------------
[1]
 size=5
 act_function=<function sigmoid at 0x0AC6E7C0>
 activations=[[0. 0. 0. 0. 0.]]
 biases=[[1 1 1 1 1]]
 weights:
     |1 1 1 1 1 |
     |1 1 1 1 1 |

--------------------------
[2]
 size=1
 act_function=<function ident at 0x068C1220>
 activations=[[0.]]
 biases=[[0]]
 weights:
     |1 |
     |1 |
     |1 |
     |1 |
     |1 |

--------------------------


In [3]:
net.calc_output(np.array([1, 1]))

array([[4.76287063]])

In [7]:
net = Net()
net.insert_layer(0, 5, ident)
net.insert_layer(1, 7, sigmoid)
net.insert_layer(2, 7, th)
net.insert_layer(3, 5, ident)

net.init_weights(1, np.array([[1,2,3,4,5,6,7],[1,2,3,4,5,6,7],[1,2,3,4,5,6,7],[1,2,3,4,5,6,7],[1,2,3,4,5,6,7]]))
net.init_weights(2, np.array([[1,2,3,4,5,6,7],[1,2,3,4,5,6,7],[1,2,3,-4,5,6,7],[1,2,3,4,5,6,7],[1,2,3,4,5,6,7],[1,2,3,4,5,6,7],[1,2,3,4,5,6,7]]))
net.init_weights(3, np.array([[1,1,0,1,1],[1,1,1,1,1],[1,1,1,0,1],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1]]))
net.init_biases(3, np.array([[1,1,1,1,1]]))

net.print_net_config()

layers=4
[0]
 size=5
 act_function=<function ident at 0x068C1220>
 activations=[[0. 0. 0. 0. 0.]]
 biases=[[0. 0. 0. 0. 0.]]
 weights:
     empty
--------------------------
[1]
 size=7
 act_function=<function sigmoid at 0x0AC6E7C0>
 activations=[[0. 0. 0. 0. 0. 0. 0.]]
 biases=[[0. 0. 0. 0. 0. 0. 0.]]
 weights:
     |1 2 3 4 5 6 7 |
     |1 2 3 4 5 6 7 |
     |1 2 3 4 5 6 7 |
     |1 2 3 4 5 6 7 |
     |1 2 3 4 5 6 7 |

--------------------------
[2]
 size=7
 act_function=<function th at 0x0AC6E808>
 activations=[[0. 0. 0. 0. 0. 0. 0.]]
 biases=[[0. 0. 0. 0. 0. 0. 0.]]
 weights:
     |1 2 3 4 5 6 7 |
     |1 2 3 4 5 6 7 |
     |1 2 3 -4 5 6 7 |
     |1 2 3 4 5 6 7 |
     |1 2 3 4 5 6 7 |
     |1 2 3 4 5 6 7 |
     |1 2 3 4 5 6 7 |

--------------------------
[3]
 size=5
 act_function=<function ident at 0x068C1220>
 activations=[[0. 0. 0. 0. 0.]]
 biases=[[1 1 1 1 1]]
 weights:
     |1 1 0 1 1 |
     |1 1 1 1 1 |
     |1 1 1 0 1 |
     |1 1 1 1 1 |
     |1 1 1 1 1 |
     |1 1 1 1 1 |
  

In [8]:
net.calc_output(np.array([1,2,0,2,1]))

array([[7.99999833, 7.99999833, 7.        , 6.99999833, 7.99999833]])

### Fill activation derivatives array

In [9]:
def calc_activations_derivatives(net: Net, target_vector):
    last_layer = net.layers[net.layers_count - 1]
    activation_der_array = []
    last_layer_derivatives_array = []

    for i in range(last_layer.neuron_count):
        der = predict_error_der(last_layer.activations, target_vector, i)
        last_layer_derivatives_array.append(der)

    activation_der_array.insert(0, last_layer_derivatives_array)

    # cal derivatives on each layer, except last and first layers
    cur_iteration = 0
    for i in range(net.layers_count - 2, 0, -1):
        cur_layer = net.layers[i]
        layer_derivatives_array = []

        for j in range(cur_layer.neuron_count):
            next_layer_der_array = activation_der_array[0]
            layer_derivatives_array.append(net.der_cost_act(i, j, next_layer_der_array))

        # add derivatives array of the current layer
        activation_der_array.insert(0, layer_derivatives_array)
        cur_iteration += 1

    return activation_der_array

array([list([1, 1]), list([1]), list([1, 1, 1])], dtype=object)

## Net training functions

In [ ]:
def get_weight_gradient_matrix(net: Net, layer_idx, activation_der_array):
    assert 0 < layer_idx < net.layers_count
    prev_layer_neuron_count = net.layers[layer_idx - 1]
    cur_layer_neuron_count = net.layers[layer_idx]

    gradient_matrix = np.zeros((prev_layer_neuron_count, cur_layer_neuron_count))

    for i in range(prev_layer_neuron_count):
        for j in range(len(cur_layer_neuron_count)):
            gradient_matrix[i][j] = net.der_cost_weigh(layer_idx, i, j, activation_der_array[layer_idx - 1][j])
    return gradient_matrix

def get_bias_gradient_matrix(net: Net, layer_idx, activation_der_array):
    assert 0 < layer_idx < net.layers_count
    cur_layer_neuron_count = net.layers[layer_idx]
    gradient_vector = np.zeroes(cur_layer_neuron_count)

    for j in range(cur_layer_neuron_count):
        gradient_vector[j] = net.der_cost_bias(layer_idx, j, activation_der_array[layer_idx - 1][j])

    return gradient_vector

def step(net: Net, weight_grad_matrices, biases_grad_matrices):
    for i in range(net.layers_count - 1):
        idx = i + 1
        cur_layer = net.layers[idx]
        cur_layer.weights += weight_grad_matrices[idx]
        cur_layer.biases += biases_grad_matrices[idx]

def training_iteration(training_data_item, target_vector, net: Net):
    net.calc_output(training_data_item)
    act_der_array = calc_activations_derivatives(net, target_vector)

    weight_grad_matrices = []
    biases_grad_matrices = []

    for i in range(net.layers_count - 1):
        idx = i + 1
        weight_grad_matrices.append(get_weight_gradient_matrix(net, idx, act_der_array))
        biases_grad_matrices.append(get_bias_gradient_matrix(net, idx, act_der_array))
    # change net parameters
    step(net, weight_grad_matrices, biases_grad_matrices)

def calc_metric(net: Net, val_target_vectors):
    predict_errors = []

    for item in val_target_vectors:
        last_layer = net.layers[net.layers_count - 1]
        predict_errors = np.append(predict_errors, predict_error(last_layer.activations, item))

    return np.mean(predict_errors)

# net training on all training data items (returns metric on validation)
def training(net: Net, training_data, target_vectors, val_target_vectors):
    for training_item, target_vector in zip(training_data, target_vectors):
        training_iteration(training_item, target_vector, net)
    return calc_metric(net, val_target_vectors)

# init weights and biases with start values
def init_net_parameters(net: Net):
    for i in range(net.layers_count - 1):
        idx = i + 1
        w_shape = net.layers[idx].weights.shape
        b_shape = net.layers[idx].biases.shape

        net.layers[idx].weights = rand.randn(w_shape[0], w_shape[1])
        net.layers[idx].biases = rand.randn(b_shape[0], b_shape[1])

## generation of training data

In [4]:
periods_count = 100
epsilon = 0.1
st_points_count = 10

dst_dir = "../data_sets/"
training_data_file = open(dst_dir + "cos_values_data", "w")

# 1 - write extremum points
training_data_file.write(str(0) + " " + str(1) + "\n")

for i in range(periods_count - 1):
    x = (np.pi / 2) * (i + 1)

    val1 = np.cos(x)
    val2 = np.cos(-x)
    training_data_file.write(str(x) + " " + str(val1) + "\n" + str(-x) + " " + str(val2) + "\n")

# write other points
st_points = np.linspace(epsilon, np.pi / 2 - epsilon, st_points_count)

for st_point in st_points:
    for i in range(periods_count):
        x = st_point + (np.pi / 2) * i
        val1 = np.cos(x)
        val2 = np.cos(-x)
        training_data_file.write(str(x) + " " + str(val1) + "\n" + str(-x) + " " + str(val2) + "\n")

training_data_file.close()

## Load training data

In [11]:
training_data_file = open(dst_dir + "cos_values_data", "r")
training_data = np.array([])
target_vectors = np.array([])
lines = []
items_count = 0

# read lines and shuffle it
for line in training_data_file:
    lines.append(line)

print("before shuffle: " + str(lines[0:5]))
rd.shuffle(lines)
print("after shuffle: " + str(lines[0:5]))
print("-------------------------")

# get training data items and target vectors:
for line in lines:
    arg, value = line.split()
    arg = float(arg)
    value = float(value)

    training_data = np.append(training_data, arg)
    target_vectors = np.append(target_vectors, value)
    items_count += 1

# reshape data to array of 1-d vectors
training_data = training_data.reshape((items_count, 1))
target_vectors = target_vectors.reshape((items_count, 1))

print("first 5 training items: " + str(training_data[0:5]))
print("first 5 training target vectors: " + str(target_vectors[0:5]))

before shuffle: ['0 1\n', '1.5707963267948966 6.123233995736766e-17\n', '-1.5707963267948966 6.123233995736766e-17\n', '3.141592653589793 -1.0\n', '-3.141592653589793 -1.0\n']
after shuffle: ['78.53981633974483 -1.0\n', '-55.83942495270743 0.7588552193149354\n', '88.82614781540026 0.6512593616359621\n', '136.40696972817878 -0.24964216559203725\n', '154.95190424376312 -0.5285844158620803\n']
-------------------------
first 5 training items: [[ 78.53981634]
 [-55.83942495]
 [ 88.82614782]
 [136.40696973]
 [154.95190424]]
first 5 training target vectors: [[-1.        ]
 [ 0.75885522]
 [ 0.65125936]
 [-0.24964217]
 [-0.52858442]]


## Split training data on batches

In [16]:
batch_size = 100
data_size = len(training_data)
batch_count = data_size // batch_size
rem = data_size % batch_size
training_data_batches = []
target_vectors_batches = []

if data_size % batch_size != 0:
    batch_count += 1

for i in range(batch_count):
    pos = i * batch_size
    tr_data_batch = []
    t_vectors_batch = []

    if i == batch_count - 1:
        tr_data_batch = training_data[pos:]
        t_vectors_batch = target_vectors[pos:]
    else:
        tr_data_batch = training_data[pos:pos + batch_size]
        t_vectors_batch = target_vectors[pos:pos + batch_size]
    training_data_batches.append(tr_data_batch)
    target_vectors_batches.append(t_vectors_batch)

print("batches' count: " + str(batch_count))
print("---------")
idx = 1
print("training data batch " + str(idx) + ":\n" + str(training_data_batches[idx]))
print("size=" + str(len(training_data_batches[idx])))

print("---------")
print("target vectors batch " + str(idx) + ":\n" + str(target_vectors_batches[idx]))
print("size=" + str(len(target_vectors_batches[idx])))

batches' count: 22
---------
training data batch 1:
[[-151.04875808]
 [ 148.97334034]
 [-117.70972451]
 [ 109.24650006]
 [  47.0238898 ]
 [ 102.00176124]
 [ -34.80982989]
 [-102.00176124]
 [ -98.25092578]
 [ 132.19920215]
 [ -69.11503838]
 [ -84.26606954]
 [  11.40019569]
 [  86.49379797]
 [ 118.06203521]
 [-143.95632996]
 [   8.25860304]
 [ -17.8356917 ]
 [  27.86971248]
 [  23.6619449 ]
 [  60.09488182]
 [ -72.35663103]
 [ -10.99557429]
 [ 112.84502483]
 [-136.10234832]
 [ 142.53784433]
 [ 119.38052084]
 [  25.99429474]
 [  88.0645943 ]
 [  49.40392894]
 [ -25.84198404]
 [ -23.3096342 ]
 [  52.08858949]
 [ 100.63096491]
 [-154.03804003]
 [ -67.13962065]
 [   7.75398163]
 [ -49.70855035]
 [ 146.94561191]
 [ 155.76114706]
 [ -14.54178835]
 [  20.82497365]
 [  93.08160469]
 [ 134.37924129]
 [ -68.25348486]
 [  -3.85083547]
 [ 101.08789702]
 [-109.24650006]
 [ -55.99173566]
 [ -31.66823724]
 [  74.07973806]
 [   0.70924281]
 [ 120.39438505]
 [  18.84955592]
 [ -10.59095288]
 [-148.821029